## Insilico digest preparation
Download of the reference data from the UCSC and setting up a BSgenome package for each species for the analysis of the insilico RRBS  
Author: Daria Romanovskaia dromanovskaia@cemm.at  
Date: 22.12.2020 (upd. 1.02.2021, upd 18.10.2021)

In [1]:
source(file.path(Sys.getenv("CODEBASE"),"DNAmeth500species/src/00.0_init.R"))

Warning message:
“package ‘ggrepel’ was built under R version 3.6.3”
Warning message:
“package ‘ggseqlogo’ was built under R version 3.6.3”
Warning message:
“package ‘pheatmap’ was built under R version 3.6.3”
Joining, by = "species"



In [2]:
library(rtracklayer)

Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, wh

## Set of species to analyse:  

We want to run the insilico RRBS on all the genomes, we used for the cross-mapping and expand by other possible maches.We have defined a set of speices, that are of potential interest for us as they have avaliable genomes + needed annotation tracks.
First version of the validation_species contained only the species name and the class and was further annotated along this notebook

In [3]:
getwd()

[1] "/home/lv71484/droman/DNAmeth500species/src/validation"

species from cross-mapping:

In [4]:
df_crossmap <- fread(file.path(analysis_dir,"validation", "01_crossMapping", "species_ucsc_matches.tsv"))
df_crossmap <- unique(df_crossmap[!is.na(ucsc_db), c("ucsc_species", "ucsc_db", "class")])

In [5]:
df_crossmap[, downloaded:= TRUE,]

In [6]:
head(df_crossmap)

ucsc_species,ucsc_db,class,downloaded
<chr>,<chr>,<chr>,<lgl>
Alpaca,vicPac2,Mammalia,TRUE
Armadillo,dasNov3,Malacostraca,TRUE
Stickleback,gasAcu1,Actinopteri,TRUE
Tasmanian devil,sarHar1,Mammalia,TRUE
Lizard,anoCar2,Reptilia,TRUE
Budgerigar,melUnd1,Aves,TRUE


In [7]:
NROW(df_crossmap)

[1] 66

species of interest:

In [8]:
df <- read.csv("../../validation/meta/validation_species.csv", sep = ";")
head(df)

,species,ucsc_genome,class,scientific_name
,<fct>,<fct>,<fct>,<fct>
1,Human,hg38,Mammalia,Homo sapiens
2,Mouse,mm10,Mammalia,Mus musculus
3,Zebrafish,danRer11,Actinopteri,Danio rerio
4,Lancelet,braFlo1,Invertebrata,Cephalochordata
5,Sea Hare,aplCal1,Invertebrata,Aplysia californica
6,Lamprey,petMar2,Jawless Vertebrate,Petromyzon marinus


In [9]:
df_full <- full_join(df_crossmap, df, by = c("ucsc_db" = "ucsc_genome", "class" = "class" ))

In [10]:
NROW(df_full)

[1] 81

In [11]:
setDT(df_full)

In [12]:
df_full[species != ucsc_species]

ucsc_species,ucsc_db,class,downloaded,species,scientific_name
<chr>,<chr>,<chr>,<lgl>,<fct>,<fct>
Budgerigar,melUnd1,Aves,TRUE,Budgeriger,Melopsittacus undulatus
Naked mole-rat,hetGla2,Mammalia,TRUE,Naked mole rat,Heterocephalus glaber
X. tropicalis,xenTro9,Amphibia,TRUE,Tropical clawed frog,Xenopus tropicalis


-> all the names make sense

In [13]:
head(df_full)

ucsc_species,ucsc_db,class,downloaded,species,scientific_name
<chr>,<chr>,<chr>,<lgl>,<fct>,<fct>
Alpaca,vicPac2,Mammalia,TRUE,NA,NA
Armadillo,dasNov3,Malacostraca,TRUE,NA,NA
Stickleback,gasAcu1,Actinopteri,TRUE,NA,NA
Tasmanian devil,sarHar1,Mammalia,TRUE,NA,NA
Lizard,anoCar2,Reptilia,TRUE,Lizard,Anolis carolinensis
Budgerigar,melUnd1,Aves,TRUE,Budgeriger,Melopsittacus undulatus


In [14]:
df_full[is.na(ucsc_species),ucsc_species:=species, by = row.names(df_full[is.na(ucsc_species)])]

In [15]:
wd <- file.path(analysis_dir,"validation", "02_insilico_digest")
dir.create(wd)
setwd(wd)

Warning message in dir.create(wd):
“'/binfl/lv71484/droman/DNAmeth500species//results_analysis/validation/02_insilico_digest' already exists”


## Download the genome tracks for every selected genome:
Takes a while

In [16]:
output_dir <- file.path(data_dir, "resources", "reference_genomes")

In [18]:
getTrack <- function(UCSC_session, track_id, dir_out, genome_id){
    my_track <- track(ucscTableQuery(UCSC_session, track=UCSCnames[track_id]))
    export.bed(my_track, file.path(dir_out, 
                    paste0(genome_id, "_", UCSCnames[track_id],".bed")))
}

In [37]:
for (i in c(78,79,80)){
    print(as.character(df_full[i]$ucsc_species))
    #where to save tracks
    dir_out <- file.path(output_dir, df_full[i]$ucsc_db,  "tracks")
    dir.create(dir_out, recursive = TRUE)
    
    #new rtracklayer seccion
    mySession <- browserSession()
    genome(mySession) <- df_full[i]$ucsc_db
    
    ##UCSC browser available tracks
    UCSCnames <- trackNames(ucscTableQuery(mySession))
    
    ## download CpGs:
    if(!"CpG Islands" %in% names(UCSCnames)) print(paste0("CpG islands not available for ",df_full[i]$ucsc_db))
    else getTrack(mySession, "CpG Islands", dir_out, df_full[i]$ucsc_db)
    
    ##download repeats:
    if(!"RepeatMasker" %in% names(UCSCnames)){
    print(paste0("RepeatMasker not available for ",df_full[i]$ucsc_db))
        
    if("simpleRepeat" %in% names(UCSCnames)) getTrack(mySession, "Simple Repeats", dir_out, df_full[i]$ucsc_db)
    else print("also no simpleRepeat")
    }
    else getTrack(mySession, "RepeatMasker", dir_out, df_full[i]$ucsc_db)
    
    ## gene annotation:
    possible_ids <- c("NCBI RefSeq", "RefSeq Genes","Ensembl Gene", "Other RefSeq", "AUGUSTUS", "Genscan Genes")
    k = 1
    while(!possible_ids[k] %in% names(UCSCnames) & k <= length(possible_ids)){
        k <- k+1
    }
    if(k > length(possible_ids)) print(paste0("no gene annotation available for ", df_full[i]$ucsc_db))
    else{
        getTrack(mySession, possible_ids[k], dir_out, df_full[i]$ucsc_db)
    }

   
    print(df_full[i]$ucsc_db)
}

[1] "Chimp"


Warning message in dir.create(dir_out, recursive = TRUE):
“'/binfl/lv71484/droman/DNAmeth500species//resources/reference_genomes/panTro6/tracks' already exists”


[1] "panTro6"
[1] "Bonobo"
[1] "panPan2"
[1] "Shrew"
[1] "sorAra2"


## saving which information was avaliable for which genome:

In [72]:
tracks_list <- sapply(df_full$ucsc_db, function(x) 
    gsub(".bed", "", gsub(paste0(x, "_"), "",
                      list.files(file.path(output_dir, x,  "tracks")))))

In [73]:
#where we have nothing (need to check by hand)
which(sapply(tracks_list, function(x) length(x)==0))

named integer(0)

In [74]:
##for other
colnames_tracks <- unique(unlist(tracks_list,use.names=FALSE))
tracks_present <- setNames(data.table(matrix(nrow = 0, ncol = length(colnames_tracks))),
                           colnames_tracks)

for(i in seq_along(tracks_list)){
temp_dt <- setNames(data.table(matrix(1, nrow = 1, ncol = length(tracks_list[[i]]))),
                    tracks_list[[i]])

  tracks_present <- rbind(tracks_present, temp_dt, fill = TRUE)
}

tracks_present[, ucsc_db := names(tracks_list[sapply(tracks_list, function(x) length(x) > 0)]),]


In [75]:
colnames(tracks_present)

[1] "cpgIslandExt"    "refGene"         "rmsk"            "refSeqComposite"
[5] "xenoRefGene"     "simpleRepeat"    "ucsc_db"

In [76]:
##check if we have gene annotation for all
 tracks_present[is.na(refSeqComposite) & is.na(refGene) & is.na(xenoRefGene)]$ucsc_db

character(0)

In [77]:
## repeats
 tracks_present[(is.na(rmsk) & is.na(simpleRepeat))]

cpgIslandExt,refGene,rmsk,refSeqComposite,xenoRefGene,simpleRepeat,ucsc_db
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>


In [78]:
## cpg islands
 tracks_present[is.na(cpgIslandExt)]

cpgIslandExt,refGene,rmsk,refSeqComposite,xenoRefGene,simpleRepeat,ucsc_db
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
NA,1,1,NA,NA,NA,gasAcu1
NA,NA,1,NA,1,1,macEug2
NA,NA,1,1,NA,NA,loxAfr3
NA,1,1,NA,NA,NA,strPur2
NA,1,1,NA,NA,NA,oryLat2
NA,NA,NA,NA,1,1,braFlo1
NA,NA,1,NA,1,1,macEug2


In [82]:
getwd()

[1] "/binfl/lv71484/droman/DNAmeth500species/results_analysis/validation/02_insilico_digest"

In [81]:
my_wt(tracks_present, file.path(analysis_dir, "validation","02_insilico_digest","tracks_present.tsv"))

## Rerunning the concat script to download and concat genomes

In [47]:
write.table(df_full[is.na(downloaded), c("ucsc_db", "ucsc_species")],"genomes_ucsc_todownload_for_insilico.tsv", sep = "\t", row.names = F, col.names = F, quote = F )

## Saving the table which genomes to run the insilico digestion on

In [84]:
my_wt(df_full[, c("ucsc_db", "ucsc_species", "species", "scientific_name", "class")], "genomes_to_run.tsv")

In [85]:
NROW(df_full)

[1] 81